<a href="https://colab.research.google.com/github/Syamabbas/Google-Collab/blob/main/Auto_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from datetime import timedelta

df = pd.read_excel("/content/drive/MyDrive/Google Collab/Data Cleansing Sym.xlsx")

In [ ]:
REFERENCE_DATE = pd.Timestamp("2025-12-01")

def convert_relative_time(text, ref_date=REFERENCE_DATE):
    if pd.isna(text):
        return None

    text = str(text).lower().strip()
    text = text.replace("diedit", "").strip()

    special_map = {
        "sebulan lalu": ("bulan", 1),
        "seminggu lalu": ("minggu", 1),
        "setahun lalu": ("tahun", 1)
    }

    if text in special_map:
        unit, value = special_map[text]
    else:
        match = re.search(r"(\d+)\s+(menit|jam|hari|minggu|bulan|tahun)\s+lalu", text)
        if not match:
            return None
        value = int(match.group(1))
        unit = match.group(2)

    if unit == "menit":
        return ref_date - timedelta(minutes=value)
    elif unit == "jam":
        return ref_date - timedelta(hours=value)
    elif unit == "hari":
        return ref_date - timedelta(days=value)
    elif unit == "minggu":
        return ref_date - timedelta(weeks=value)
    elif unit == "bulan":
        return ref_date - pd.DateOffset(months=value)
    elif unit == "tahun":
        return ref_date - pd.DateOffset(years=value)

    return None

In [ ]:
df['tanggal_review'] = df['Waktu'].apply(convert_relative_time)
df['tanggal_review'] = pd.to_datetime(df['tanggal_review']).dt.date

In [ ]:
typo_dict = {
# Waktu & Pengiriman
    "telattt": "telat",
    "telatt": "telat",
    "terlat": "terlambat",
    "terlmbt": "terlambat",
    "ngaret": "terlambat",
    "nungguin": "menunggu",
    "nungguu": "menunggu",
    "nunggu banget": "menunggu banget",
    "kelamaan": "lama",
    "lamanyaa": "lama",

    # Sampai / Kirim
    "sampee": "sampai",
    "sampeee": "sampai",
    "smpee": "sampai",
    "nyamppe": "sampai",
    "nyampeee": "sampai",
    "blm nyampe": "belum sampai",
    "blom": "belum",
    "blom sampai": "belum sampai",
    "gk nyampe": "tidak sampai",
    "ga nyampe": "tidak sampai",
    "belom": "belum",

    # Respon / CS / Admin
    "responn": "respon",
    "slowresp": "slow respon",
    "fastresp": "fast respon",
    "gak respon": "tidak respon",
    "ga respon": "tidak respon",
    "slow respon banget": "slow respon",
    "bales lama": "slow respon",
    "dibales lama": "slow respon",
    "chat ga dibales": "tidak dibalas",
    "chat gak dibales": "tidak dibalas",
    "cs lemot": "slow respon",
    "admin lemot": "slow respon",

    # Bagus / Positif
    "baguuus": "bagus",
    "baguuuss": "bagus",
    "mantaaaap": "mantap",
    "mantappp": "mantap",
    "mantab": "mantap",
    "mantep": "mantap",
    "mantepp": "mantap",
    "mantul banget": "mantap",
    "keren": "bagus",
    "kereen": "bagus",
    "kereenn": "bagus",
    "the bestt": "the best",
    "recommendedd": "recommended",
    "rekomen banget": "recommended",

    # Negatif / Emosi
    "paraaaah": "parah",
    "parahhh": "parah",
    "payaaaah": "payah",
    "kesel banget": "kesal",
    "nyebelin": "menyebalkan",
    "nyesel banget": "menyesal",
    "kapokkk": "kapok",
    "kapoook": "kapok",
    "bad banget": "buruk",
    "jelekkk": "jelek",
    "jelek banget": "jelek",

    # Rusak / Barang
    "rusakkk": "rusak",
    "rusakk": "rusak",
    "hancurrr": "hancur",
    "hancurr": "hancur",
    "pecahh": "pecah",
    "pecahhh": "pecah",
    "remuk": "rusak",
    "kardus ancur": "kardus rusak",

    # Hilang / Nyasar
    "ilang": "hilang",
    "ilanggg": "hilang",
    "nyasarr": "nyasar",
    "nyasarrr": "nyasar",
    "paket ilang": "paket hilang",
    "barang ilang": "barang hilang",

    # Harga / Ongkir
    "mahal bgt": "mahal banget",
    "murah bgt": "murah banget",
    "ongkirnya mahal": "ongkir mahal",
    "ongkirnya murah": "ongkir murah",
    "kemahalan": "mahal",
    "kemurahan": "murah",

    # Kurir / Driver
    "kurir galak banget": "kurir galak",
    "driver galak": "kurir galak",
    "kurir jutek banget": "kurir jutek",
    "abang kurir": "kurir",
    "pak kurir": "kurir",

    # Profanity / Kasar
    "anj": "anjing",
    "anjayy": "anjing",
    "anjay": "anjing",
    "bangke": "bangsat",
    "bangkee": "bangsat",
    "kampret": "kasar",
    "brengsek": "buruk",
    "tai banget": "tai",

    # Sistem / Aplikasi
    "aplikasinya error": "aplikasi error",
    "apps error": "aplikasi error",
    "web error": "website error",
    "tracking error": "tracking bermasalah",
    "resi ga ketemu": "resi tidak ditemukan",
    "resi gak muncul": "resi tidak ditemukan",
    "status ga update": "status tidak update",

    # Lokasi / Tempat
    "lokasii": "lokasi",
    "lokasii": "lokasi",
    "lokasinyaa": "lokasi",
    "tempatnya bersih": "tempat bersih",
    "tempatnya nyaman": "tempat nyaman",

    # Rating / Review
    "bintang 1": "bintang satu",
    "bintang1": "bintang satu",
    "rate jelek": "rating buruk",
    "review jelek": "review buruk",
    "komen jelek": "komentar buruk",

    # Lain-lain
    "makasiii": "terima kasih",
    "makasihh": "terima kasih",
    "thx": "terima kasih",
    "thanks": "terima kasih",
    "okeee": "oke",
    "okee": "oke",
    "sippp": "sip",
    "sipppp": "sip"
}

patterns = {
    r'(\d+)\s*hri': r'\1 hari',
    r'(\d+)\s*hr': r'\1 hari',
    r'(\d+)\s*d': r'\1 hari'
}

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def normalize_typo(text):
    for typo, correct in typo_dict.items():
        text = re.sub(rf'\b{typo}\b', correct, text)
    return text

def normalize_pattern(text):
    for p, rpl in patterns.items():
        text = re.sub(p, rpl, text)
    return text

In [ ]:
df['clean_text'] = df['Komentar'].apply(clean_text)
df['clean_text'] = df['clean_text'].apply(normalize_pattern)
df['clean_text'] = df['clean_text'].apply(normalize_typo)

In [ ]:
positive_words = [
    "1 hari sampai","2 hari sampai","3 hari sampai","good",
    "alhamdulillah aman","alhamdulillah sampai","alhamdulillah tepat waktu",
    "aman","aman aja","aman sampai tujuan","aman-aman aja","amanah",
    "bagus","baik","barang sampai","bersih","cepat","cepat banget",
    "cepat dan aman","cepet","efektif","efisien","extra cepat","fast respon",
    "harga bersahabat","harga bersaing","harga kirim murah","harga murah",
    "kurir ramah","lancar","langganan","lengkap","lokasi aman","lokasi mudah",
    "mantap","mantap bosku","mantap pelayanan","mantap sekali","mantul",
    "memuaskan","murah","nyaman","not bad","ok","ok juga","oke",
    "ongkir murah","ongkos tidak mahal","ontime","on time","paket sampai",
    "paling murah","pelayanan baik","pelayanan bagus","pelayanan cepat",
    "pelayanan konsisten","pelayanannya ramah","pegawai sangat ramah",
    "percaya","profesional","puas","ramah","recommended","rapi",
    "selalu aman","selalu sampai","semalam sampai","sentral cargo terbaik",
    "service bagus","sesuai","sesuai estimasi","sangat ekonomis",
    "sangat membantu","staff baik dan ramah","staff ramah","strategis",
    "tepat waktu","tercepat","terima kasih","terjangkau","tidak hilang",
    "tidak melebihi estimasi","worth it","makasih","andalan",
    "dapat diandalkan","petugas responsif","konfirmasi bagus",
    "karyawan membantu","kooperatif cepat","karyawan gesit",
    "pengiriman cepat","cargo murah","harga tidak mahal","pertahankan ya"
]

negative_words = [
    "abal-abal","alamat tidak ditemukan","amburadul","tidak konsisten",
    "aplikasi tidak update","asal","asu","asuu","bagong",
    "belum","belum sampai","berkurang","bikin emosi","bikin nyesel",
    "bocor","bosok","buruk","capek","cuek","cerewet","crewet",
    "delay","diblokir","ditahan","dibanting","lempar","edan","emosi",
    "error","ekspedisi terburuk","ga dibales","ga jelas","galak",
    "gembel","gila","hilang","harga tidak sesuai","harus bayar lagi",
    "internal buruk","jorok","jelek","kacau","kaga becus","kapok",
    "kardus hancur","kasar","kecewa","kecewakan","kerja buruk",
    "kesal","kesel","kotor","kurang","kurir galak","kurir ceroboh",
    "kurir tidak paham","kurang etika","lama","lambat","laporan palsu",
    "lelet","lemot","lemod","macet","malas","mandek","marah",
    "masalah","meluber","mengecewakan","mengeluh","menyesal",
    "merugikan","molor","muak","muter muter","nabrak","ngaret",
    "ngerepotin","nginep","no respon","not recommended","nyasar",
    "nyesel","ongkir kurang","ora jelas","paket ngendap","paling buruk",
    "paling jelek","parah","payah","pending","pikir ulang","palsu",
    "resi tidak sesuai","review ancur","ribet","rugi","rusak",
    "salah","salah kirim","salah penerima","sabaran","serempet",
    "shaming","slow respon","status palsu","stuck","stuk",
    "susah","susah dihubungi","tai","tanpa permintaan maaf","telat",
    "telepon tidak diangkat","tertahan","tidak ada kabar",
    "tidak ada komunikasi","tidak ada konfirmasi","tidak ada respon",
    "tidak amanah","tidak becus","tidak bergerak","tidak berkualitas",
    "tidak bertanggung jawab","tidak bisa klaim","tidak dikirim",
    "tidak diantar","tidak direkomendasikan","tidak hati-hati",
    "tidak jalan","tidak niat","tidak profesional","tidak responsif",
    "tidak sopan","tidak sampai","tidak sesuai jaminan",
    "tidur-tiduran","toilet kotor","tolol","transit lama",
    "volume tidak sesuai","viralkan","wa tidak aktif","waswas",
    "barang tertinggal","barang terpisah","terselip","teledor",
    "bolak balik","ugal-ugalan","perlu perbaikan","perlu training",
    "perbaiki kurir","perbaiki admin","bintang satu","acuh",
    "attitude buruk","arogan","ketus","admin galak","berisik"
]

In [ ]:
def auto_label(text, pos_words, neg_words):
    pos_count = 0
    neg_count = 0

    for word in pos_words:
        if word in text:
            pos_count += 1

    for word in neg_words:
        if word in text:
            neg_count += 1

    if pos_count > neg_count:
        return "positif"
    elif neg_count > pos_count:
        return "negatif"
    else:
        return "netral"

In [ ]:
df['label'] = df['clean_text'].apply(
    lambda x: auto_label(x, positive_words, negative_words)
)

df[['Komentar', 'label']]

,Komentar,label
0,"barang saya kurang 1 koli, katanya dikirim bes...",negatif
1,"Barang sudah 4 hari ngak diantar antar,ngak ad...",negatif
2,Adakah no telpon yg bisa dihubungi untuk Sentr...,netral
3,"Ekspedisi amburadul, sistem kacau, tanya posis...",negatif
4,"padahal udah langganan sama sentral,,paket dar...",positif
...,...,...
5201,Pelayanan yg baik,positif
5202,"Mau kirim motor, eh belum bisa...",negatif
5203,Bagus,positif
5204,Sentral Cargo adalah yang terbaik,positif


In [ ]:
df_ml = df[df['label'] != 'netral'].copy()

In [ ]:
def auto_label_with_score(text, pos_words, neg_words):
    pos_count = sum(1 for w in pos_words if w in text)
    neg_count = sum(1 for w in neg_words if w in text)
    score = abs(pos_count - neg_count)

    if pos_count > neg_count:
        return "positif", score
    elif neg_count > pos_count:
        return "negatif", score
    else:
        return "netral", 0

In [ ]:
df[['label','confidence']] = df['clean_text'].apply(
    lambda x: pd.Series(auto_label_with_score(x, positive_words, negative_words))
)

In [ ]:
df_ml = df[df['confidence'] >= 1]

In [ ]:
# df_ml[['clean_text','label']]
df_ml.to_excel("hasil_auto_labeling.xlsx", index=False)